In [12]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from d2l import tensorflow as d2l
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from datetime import datetime

# 1
class LinearRegressionData(d2l.DataModule):
    """ implementacao do DataModule pra dados externos
    Defined in :numref:`sec_synthetic-regression-data`"""
    
    def __init__(self,
                entrada,
                saida,
                num_train=5197,
                num_val=1300,
                batch_size=32):
        super().__init__()
        self.save_hyperparameters(['tensor'])
        self.X = entrada
        self.y = saida
# 2
    def get_dataloader(self, train):
        """Defined in :numref:`sec_synthetic-regression-data`"""
        i = slice(0, self.num_train) if train else slice(self.num_train, None)
        return self.get_tensorloader((self.X, self.y), train, i)

In [23]:
# Nome do arquivo e onde está localizado
arquivo_csv = os.path.expanduser('Base de Dados.csv')

data = pd.read_csv(arquivo_csv)
data

,Data,Pontuacao,Tipo_Coleta,Sexo,Fx_etaria,Fx_renda,Presidencia,Governo,Turno,Hora
0,2022-08-04 18:58:49 UTC,0,AAS sistemática,Feminino,19 a 24,1 a 2 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/não quero responder,Noturno,18:58:49
1,2022-08-04 19:00:30 UTC,0,AAS sistemática,Masculino,25 a 29,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/não quero responder,Noturno,19:00:29
2,2022-08-04 19:01:50 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,Não sei/ não quero responder,Não sei/não quero responder,Noturno,19:01:49
3,2022-08-04 19:02:40 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Leandro Grass (PV),Noturno,19:02:39
4,2022-08-04 19:04:02 UTC,0,AAS sistemática,Masculino,19 a 24,mais de 10 SM,SIMONE TEBET (MDB),Leandro Grass (PV),Noturno,19:04:02
...,...,...,...,...,...,...,...,...,...,...
195,2022-08-01 16:06:58 UTC,0,AAS sistemática,Feminino,19 a 24,Até 1 SM,VERA LÚCIA (PSTU),Não sei/ Não quero responder,Diurno,16:06:58
196,2022-08-02 08:46:31 UTC,0,AAS sistemática,Feminino,19 a 24,5 a 10 SM,CIRO GOMES (PDT),Keka Bagno (PSOL),Diurno,08:46:31
197,2022-07-26 11:20:43 UTC,0,AAS sistemática,Feminino,19 a 24,Sem Resposta,Não sei/ Não quero responder,Não sei/ Não quero responder,Diurno,11:20:43
198,2022-08-02 09:10:19 UTC,0,AAS sistemática,Feminino,19 a 24,mais de 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Keka Bagno (PSOL),Diurno,09:10:19


In [14]:
opcoes_possiveis = data['Presidencia'].unique()
print(opcoes_possiveis)


['LUIZ INÁCIO LULA DA SILVA (PT)' 'Não sei/ não quero responder'
 'SIMONE TEBET (MDB)' 'CIRO GOMES (PDT)' 'JAIR MESSIAS BOLSONARO (PL)'
 'Nulo' 'Branco' 'José Maria Eymael (DC)' 'Não sei/ Não quero responder'
 'ANDRÉ JANONES ( Avante )' "FELIPE D'AVILA (Novo)" 'SOFIA MANZANO (PCB)'
 'LEONARDO PÉRICLES (Unidade Popular)' 'VERA LÚCIA (PSTU)']


In [15]:
opcoes_possiveis = data['Governo'].unique()
print(opcoes_possiveis)

['Não sei/não quero responder' 'Leandro Grass (PV)' 'Ibanes Rocha (MDB)'
 'Branco' 'José Antônio Reguffe (União Brasil)' 'Leila Barros (PDT)'
 'Rafael Parente (PSB)' 'Paula Belmonte (Cidadania)'
 'Robson da Silva (PSTU)' 'Keka Bagno (PSOL)' 'Izalci Lucas (PSDB)' 'Nulo'
 'Não sei/ Não quero responder' 'Ibaneis Rocha (MDB)' 'Lucas Salles (DC)']


In [16]:
# Escritos errados ou duplicado

novos = ['Não sei/ Não quero responder', 'Ibaneis Rocha (MDB)' ] # --> Vão ser substituidos
errados = ['Não sei/não quero responder', 'Ibanes Rocha (MDB)'] # --> Escritos errados

for i in range(len(novos)):
    data['Governo'] = data['Governo'].replace(errados[i], novos[i])

# Trocados para Esquerda, Direita, Centro-Esquerda, Centro-Direita, Centro-Esquerda
governadores = ['Leandro Grass (PV)', 'Ibaneis Rocha (MDB)', 'José Antônio Reguffe (União Brasil)', 'Leila Barros (PDT)', 'Rafael Parente (PSB)', 
                'Paula Belmonte (Cidadania)', 'Robson da Silva (PSTU)', 'Robson da Silva (PSTU)', 'Keka Bagno (PSOL)', 'Keka Bagno (PSOL)', 'Lucas Salles (DC)','Izalci Lucas (PSDB)']

for i in range(len(governadores)):
    # Esquerda
    if governadores[i] == 'Leandro Grass (PV)' or governadores[i] == 'Keka Bagno (PSOL)' or governadores[i] == 'Robson da Silva (PSTU)' or governadores[i] == 'Paula Belmonte (Cidadania)':
        data['Governo'] = data['Governo'].replace(governadores[i], 'Esquerda')
        
    # Direita
    if governadores[i] == 'Lucas Salles (DC)':
        data['Governo'] = data['Governo'].replace(governadores[i], 'Direita')
        
    # Centro-Esquerda
    if governadores[i] == 'Leila Barros (PDT)' or governadores[i] == 'Rafael Parente (PSB)':
        data['Governo'] = data['Governo'].replace(governadores[i], 'Centro-Esquerda')
    
    # Centro-Direita
    if governadores[i] == 'Ibaneis Rocha (MDB)' or governadores[i] == 'José Antônio Reguffe (União Brasil)' or governadores[i] == 'Izalci Lucas (PSDB)':
        data['Governo'] = data['Governo'].replace(governadores[i], 'Centro-Direita')

data
    

,Data,Pontuacao,Tipo_Coleta,Sexo,Fx_etaria,Fx_renda,Presidencia,Governo,Turno,Hora
0,2022-08-04 18:58:49 UTC,0,AAS sistemática,Feminino,19 a 24,1 a 2 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Noturno,18:58:49
1,2022-08-04 19:00:30 UTC,0,AAS sistemática,Masculino,25 a 29,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Noturno,19:00:29
2,2022-08-04 19:01:50 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,Não sei/ não quero responder,Não sei/ Não quero responder,Noturno,19:01:49
3,2022-08-04 19:02:40 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Esquerda,Noturno,19:02:39
4,2022-08-04 19:04:02 UTC,0,AAS sistemática,Masculino,19 a 24,mais de 10 SM,SIMONE TEBET (MDB),Esquerda,Noturno,19:04:02
...,...,...,...,...,...,...,...,...,...,...
195,2022-08-01 16:06:58 UTC,0,AAS sistemática,Feminino,19 a 24,Até 1 SM,VERA LÚCIA (PSTU),Não sei/ Não quero responder,Diurno,16:06:58
196,2022-08-02 08:46:31 UTC,0,AAS sistemática,Feminino,19 a 24,5 a 10 SM,CIRO GOMES (PDT),Esquerda,Diurno,08:46:31
197,2022-07-26 11:20:43 UTC,0,AAS sistemática,Feminino,19 a 24,Sem Resposta,Não sei/ Não quero responder,Não sei/ Não quero responder,Diurno,11:20:43
198,2022-08-02 09:10:19 UTC,0,AAS sistemática,Feminino,19 a 24,mais de 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Esquerda,Diurno,09:10:19


In [17]:
opcoes_possiveis = data['Governo'].unique()
print(opcoes_possiveis)

['Não sei/ Não quero responder' 'Esquerda' 'Centro-Direita' 'Branco'
 'Centro-Esquerda' 'Nulo' 'Direita']


In [18]:
data['Hora'] = pd.to_datetime(data['Hora'], format='%H:%M:%S').dt.time

def round_to_nearest_hour(tm):
    return tm.replace(minute=0, second=0, microsecond=0)
data['Hora'] = data['Hora'].apply(round_to_nearest_hour)


data


,Data,Pontuacao,Tipo_Coleta,Sexo,Fx_etaria,Fx_renda,Presidencia,Governo,Turno,Hora
0,2022-08-04 18:58:49 UTC,0,AAS sistemática,Feminino,19 a 24,1 a 2 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Noturno,18:00:00
1,2022-08-04 19:00:30 UTC,0,AAS sistemática,Masculino,25 a 29,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Noturno,19:00:00
2,2022-08-04 19:01:50 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,Não sei/ não quero responder,Não sei/ Não quero responder,Noturno,19:00:00
3,2022-08-04 19:02:40 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Esquerda,Noturno,19:00:00
4,2022-08-04 19:04:02 UTC,0,AAS sistemática,Masculino,19 a 24,mais de 10 SM,SIMONE TEBET (MDB),Esquerda,Noturno,19:00:00
...,...,...,...,...,...,...,...,...,...,...
195,2022-08-01 16:06:58 UTC,0,AAS sistemática,Feminino,19 a 24,Até 1 SM,VERA LÚCIA (PSTU),Não sei/ Não quero responder,Diurno,16:00:00
196,2022-08-02 08:46:31 UTC,0,AAS sistemática,Feminino,19 a 24,5 a 10 SM,CIRO GOMES (PDT),Esquerda,Diurno,08:00:00
197,2022-07-26 11:20:43 UTC,0,AAS sistemática,Feminino,19 a 24,Sem Resposta,Não sei/ Não quero responder,Não sei/ Não quero responder,Diurno,11:00:00
198,2022-08-02 09:10:19 UTC,0,AAS sistemática,Feminino,19 a 24,mais de 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Esquerda,Diurno,09:00:00


In [19]:
data['Hora'] = pd.to_datetime(data['Hora'], format='%H:%M:%S').dt.time

def round_to_nearest_hour(tm):
    return tm.replace(minute=0, second=0, microsecond=0)

data['Hora'] = data['Hora'].apply(round_to_nearest_hour)

periodos = {
    (0, 12): 'Manhã',
    (13, 17): 'Tarde',
    (18, 23): 'Noite'
}

def obter_periodo(hora):
    for horas, periodo in periodos.items():
        if horas[0] <= hora.hour <= horas[1]:
            return periodo

data['Periodo'] = data['Hora'].apply(obter_periodo)
data


,Data,Pontuacao,Tipo_Coleta,Sexo,Fx_etaria,Fx_renda,Presidencia,Governo,Turno,Hora,Periodo
0,2022-08-04 18:58:49 UTC,0,AAS sistemática,Feminino,19 a 24,1 a 2 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Noturno,18:00:00,Noite
1,2022-08-04 19:00:30 UTC,0,AAS sistemática,Masculino,25 a 29,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Noturno,19:00:00,Noite
2,2022-08-04 19:01:50 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,Não sei/ não quero responder,Não sei/ Não quero responder,Noturno,19:00:00,Noite
3,2022-08-04 19:02:40 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Esquerda,Noturno,19:00:00,Noite
4,2022-08-04 19:04:02 UTC,0,AAS sistemática,Masculino,19 a 24,mais de 10 SM,SIMONE TEBET (MDB),Esquerda,Noturno,19:00:00,Noite
...,...,...,...,...,...,...,...,...,...,...,...
195,2022-08-01 16:06:58 UTC,0,AAS sistemática,Feminino,19 a 24,Até 1 SM,VERA LÚCIA (PSTU),Não sei/ Não quero responder,Diurno,16:00:00,Tarde
196,2022-08-02 08:46:31 UTC,0,AAS sistemática,Feminino,19 a 24,5 a 10 SM,CIRO GOMES (PDT),Esquerda,Diurno,08:00:00,Manhã
197,2022-07-26 11:20:43 UTC,0,AAS sistemática,Feminino,19 a 24,Sem Resposta,Não sei/ Não quero responder,Não sei/ Não quero responder,Diurno,11:00:00,Manhã
198,2022-08-02 09:10:19 UTC,0,AAS sistemática,Feminino,19 a 24,mais de 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Esquerda,Diurno,09:00:00,Manhã


In [20]:
colunas_para_apagar = ['Data', 'Pontuacao', 'Tipo_Coleta', 'Pontuacao','Hora'] # --> Coloca aqui os nomes das colunas para APAGAR
#Prestar atenção a HORA

data = data.drop(colunas_para_apagar, axis=1, errors='ignore')
data

,Sexo,Fx_etaria,Fx_renda,Presidencia,Governo,Turno,Periodo
0,Feminino,19 a 24,1 a 2 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Noturno,Noite
1,Masculino,25 a 29,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Noturno,Noite
2,Masculino,19 a 24,5 a 10 SM,Não sei/ não quero responder,Não sei/ Não quero responder,Noturno,Noite
3,Masculino,19 a 24,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Esquerda,Noturno,Noite
4,Masculino,19 a 24,mais de 10 SM,SIMONE TEBET (MDB),Esquerda,Noturno,Noite
...,...,...,...,...,...,...,...
195,Feminino,19 a 24,Até 1 SM,VERA LÚCIA (PSTU),Não sei/ Não quero responder,Diurno,Tarde
196,Feminino,19 a 24,5 a 10 SM,CIRO GOMES (PDT),Esquerda,Diurno,Manhã
197,Feminino,19 a 24,Sem Resposta,Não sei/ Não quero responder,Não sei/ Não quero responder,Diurno,Manhã
198,Feminino,19 a 24,mais de 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Esquerda,Diurno,Manhã


In [25]:
# Escritos errados ou duplicado

for i in range(1):
    data['Presidencia'] = data['Presidencia'].replace('Não sei/ não quero responder', 'Não sei/ Não quero responder')
    
Presidentes = ['SIMONE TEBET (MDB)', 'CIRO GOMES (PDT)', 'José Maria Eymael (DC)', 'ANDRÉ JANONES ( Avante )', "FELIPE D'AVILA (Novo)", 'SOFIA MANZANO (PCB)','LEONARDO PÉRICLES (Unidade Popular)','VERA LÚCIA (PSTU)' ]

for i in range(len(Presidentes)):
    # Lula
    if Presidentes[i] == 'SIMONE TEBET (MDB)' or Presidentes[i] == 'CIRO GOMES (PDT)' or Presidentes[i] == 'Robson da Silva (PSTU)' or Presidentes[i] == 'Paula Belmonte (Cidadania)' or Presidentes[i] == 'SOFIA MANZANO (PCB)' or Presidentes[i] == 'VERA LÚCIA (PSTU)' or Presidentes[i] == 'LEONARDO PÉRICLES (Unidade Popular)' or Presidentes[i] == 'ANDRÉ JANONES ( Avante )':
        data['Presidencia'] = data['Presidencia'].replace(Presidentes[i], 'LUIZ INÁCIO LULA DA SILVA (PT)')
    # Bolsonaro
    if Presidentes[i] == 'José Maria Eymael (DC)' or Presidentes[i] == "FELIPE D'AVILA (Novo)":
        data['Presidencia'] = data['Presidencia'].replace(Presidentes[i], 'JAIR MESSIAS BOLSONARO (PL)')

data
    


,Data,Pontuacao,Tipo_Coleta,Sexo,Fx_etaria,Fx_renda,Presidencia,Governo,Turno,Hora
0,2022-08-04 18:58:49 UTC,0,AAS sistemática,Feminino,19 a 24,1 a 2 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/não quero responder,Noturno,18:58:49
1,2022-08-04 19:00:30 UTC,0,AAS sistemática,Masculino,25 a 29,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/não quero responder,Noturno,19:00:29
2,2022-08-04 19:01:50 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,Não sei/ Não quero responder,Não sei/não quero responder,Noturno,19:01:49
3,2022-08-04 19:02:40 UTC,0,AAS sistemática,Masculino,19 a 24,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Leandro Grass (PV),Noturno,19:02:39
4,2022-08-04 19:04:02 UTC,0,AAS sistemática,Masculino,19 a 24,mais de 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Leandro Grass (PV),Noturno,19:04:02
...,...,...,...,...,...,...,...,...,...,...
195,2022-08-01 16:06:58 UTC,0,AAS sistemática,Feminino,19 a 24,Até 1 SM,LUIZ INÁCIO LULA DA SILVA (PT),Não sei/ Não quero responder,Diurno,16:06:58
196,2022-08-02 08:46:31 UTC,0,AAS sistemática,Feminino,19 a 24,5 a 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Keka Bagno (PSOL),Diurno,08:46:31
197,2022-07-26 11:20:43 UTC,0,AAS sistemática,Feminino,19 a 24,Sem Resposta,Não sei/ Não quero responder,Não sei/ Não quero responder,Diurno,11:20:43
198,2022-08-02 09:10:19 UTC,0,AAS sistemática,Feminino,19 a 24,mais de 10 SM,LUIZ INÁCIO LULA DA SILVA (PT),Keka Bagno (PSOL),Diurno,09:10:19


In [26]:
opcoes_possiveis = data['Presidencia'].unique()
print(opcoes_possiveis)

['LUIZ INÁCIO LULA DA SILVA (PT)' 'Não sei/ Não quero responder'
 'JAIR MESSIAS BOLSONARO (PL)' 'Nulo' 'Branco']


In [22]:
data = data[['Sexo', 'Fx_etaria', 'Fx_renda', 'Governo', 'Turno', 'Periodo', 'Presidencia']]
data

,Sexo,Fx_etaria,Fx_renda,Governo,Turno,Periodo,Presidencia
0,Feminino,19 a 24,1 a 2 SM,Não sei/ Não quero responder,Noturno,Noite,LUIZ INÁCIO LULA DA SILVA (PT)
1,Masculino,25 a 29,5 a 10 SM,Não sei/ Não quero responder,Noturno,Noite,LUIZ INÁCIO LULA DA SILVA (PT)
2,Masculino,19 a 24,5 a 10 SM,Não sei/ Não quero responder,Noturno,Noite,Não sei/ Não quero responder
3,Masculino,19 a 24,5 a 10 SM,Esquerda,Noturno,Noite,LUIZ INÁCIO LULA DA SILVA (PT)
4,Masculino,19 a 24,mais de 10 SM,Esquerda,Noturno,Noite,LUIZ INÁCIO LULA DA SILVA (PT)
...,...,...,...,...,...,...,...
195,Feminino,19 a 24,Até 1 SM,Não sei/ Não quero responder,Diurno,Tarde,LUIZ INÁCIO LULA DA SILVA (PT)
196,Feminino,19 a 24,5 a 10 SM,Esquerda,Diurno,Manhã,LUIZ INÁCIO LULA DA SILVA (PT)
197,Feminino,19 a 24,Sem Resposta,Não sei/ Não quero responder,Diurno,Manhã,Não sei/ Não quero responder
198,Feminino,19 a 24,mais de 10 SM,Esquerda,Diurno,Manhã,LUIZ INÁCIO LULA DA SILVA (PT)
